# This file extracts feature sets from pcap files.

### Input & Output

`Input Files`: All files with the pcap extension in the “./pcaps/” folder is read.

`Output Files`: Fingerprint result file named *FP_MAIN.csv*.

###  importing relevant libraries

In [1]:
from scapy.all import*
import math
import pandas as pd
import os
import numpy as np


### Discovering pcap extension files under "pcaps" folder.

In [2]:
def find_the_way(path,file_format):
    count=0
    files_add = []
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                files_add.append(os.path.join(r,file))  
    return files_add
files_add=find_the_way('./pcaps/','.pcap')

### List of pcap files to be processed

In [3]:
files_add

['./pcaps/HueSwitch/Setup-C-8-STA.pcap',
 './pcaps/HueSwitch/Setup-C-9-STA.pcap',
 './pcaps/HueSwitch/Setup-C-2-STA.pcap',
 './pcaps/HueSwitch/Setup-C-3-STA.pcap',
 './pcaps/HueSwitch/Setup-C-10-STA.pcap',
 './pcaps/HueSwitch/Setup-A-4-STA.pcap',
 './pcaps/HueSwitch/Setup-A-5-STA.pcap',
 './pcaps/HueSwitch/Setup-A-9-STA.pcap',
 './pcaps/HueSwitch/Setup-A-8-STA.pcap',
 './pcaps/HueSwitch/Setup-A-3-STA.pcap',
 './pcaps/HueSwitch/Setup-A-2-STA.pcap',
 './pcaps/HueSwitch/Setup-C-5-STA.pcap',
 './pcaps/HueSwitch/Setup-C-4-STA.pcap',
 './pcaps/HueSwitch/Setup-A-7-STA.pcap',
 './pcaps/HueSwitch/Setup-A-6-STA.pcap',
 './pcaps/HueSwitch/Setup-C-1-STA.pcap',
 './pcaps/HueSwitch/Setup-C-6-STA.pcap',
 './pcaps/HueSwitch/Setup-C-7-STA.pcap',
 './pcaps/HueSwitch/Setup-A-1-STA.pcap',
 './pcaps/HueSwitch/Setup-A-10-STA.pcap',
 './pcaps/D-LinkHomeHub/Setup-C-8-STA.pcap',
 './pcaps/D-LinkHomeHub/Setup-C-9-STA.pcap',
 './pcaps/D-LinkHomeHub/Setup-C-2-STA.pcap',
 './pcaps/D-LinkHomeHub/Setup-C-3-STA.pcap'

### Port numbers are classified in this part as:

| Port Numbers | Equivalents |
| :------ | :------ |
|No port| 0|
|Well known ports (between 0 and 1023) |1|
|Rregistered ports (between 1024 and 49151)  |2|
|Dynamic ports (between  49152 and 65535) |3|
# ↓ 

In [4]:
def port_class(port):
    if 0 <= port <= 1023:
        return 1
    elif  1024 <= port <= 49151 :
        return 2
    elif 49152 <=port <= 65535 :
        return 3
    else:
        return 0

### The dictionary to be used for MAC address and device matching.
#### Datasets, their MAC addresses and Devices are given separately.

In [5]:
MAC_list={
# UNSW IEEE TMC 2018 Data MAC and Device names
"d0:52:a8:00:67:5e":"Smart Things",
"44:65:0d:56:cc:d3":"Amazon Echo",
"70:ee:50:18:34:43":"Netatmo Welcome",
"f4:f2:6d:93:51:f1":"TP-Link Day Night Cloud camera",
"00:16:6c:ab:6b:88":"Samsung SmartCam",
"30:8c:fb:2f:e4:b2":"Dropcam",
"00:62:6e:51:27:2e":"Insteon Camera",
"e8:ab:fa:19:de:4f":"unknown maybe cam",
"00:24:e4:11:18:a8":"Withings Smart Baby Monitor",
"ec:1a:59:79:f4:89":"Belkin Wemo switch",
"50:c7:bf:00:56:39":"TP-Link Smart plug",
"74:c6:3b:29:d7:1d":"iHome",
"ec:1a:59:83:28:11":"Belkin wemo motion sensor",
"18:b4:30:25:be:e4":"NEST Protect smoke alarm",
"70:ee:50:03:b8:ac":"Netatmo weather station",
"00:24:e4:1b:6f:96":"Withings Smart scale",
"74:6a:89:00:2e:25":"Blipcare Blood Pressure meter",
"00:24:e4:20:28:c6":"Withings Aura smart sleep sensor",
"d0:73:d5:01:83:08":"Light Bulbs LiFX Smart Bulb",
"18:b7:9e:02:20:44":"Triby Speaker",
"e0:76:d0:33:bb:85":"PIX-STAR Photo-frame",
"70:5a:0f:e4:9b:c0":"HP Printer",
"08:21:ef:3b:fc:e3":"Samsung Galaxy Tab",
"30:8c:fb:b6:ea:45":"Nest Dropcam",
"40:f3:08:ff:1e:da":"Android Phone",
"74:2f:68:81:69:42":"Laptop",
"ac:bc:32:d4:6f:2f":"MacBook",
"b4:ce:f6:a7:a3:c2":"Android Phone",
"d0:a6:37:df:a1:e1":"IPhone",
"f4:5c:89:93:cc:85":"MacBook/Iphone",
"14:cc:20:51:33:ea":"TPLink Router Bridge LAN (Gateway)",
# Yourthings Data MAC and Device names
'00:01:c0:18:7f:9b': 'Gateway',
 '00:04:4b:55:f6:4f': 'nVidiaShield',
 '00:12:16:ab:c0:22': 'ChineseWebcam',
 '00:17:88:21:f7:e4': 'PhilipsHUEHub',
 '00:1d:c9:23:f6:00': 'RingDoorbell',
 '00:21:cc:4d:59:35': 'Wink2Hub',
 '00:24:e4:2b:a5:34': 'WithingsHome',
 '00:7e:56:77:35:4d': 'KoogeekLightbulb',
 '08:05:81:ee:06:46': 'Roku4',
 '08:86:3b:6f:7a:15': 'BelkinWeMoMotionSensor',
 '08:86:3b:70:d7:39': 'BelkinWeMoSwitch',
 '0c:47:c9:4e:fe:5b': 'AmazonFireTV',
 '10:ce:a9:eb:5a:8a': 'BoseSoundTouch10',
 '18:b4:30:31:04:b9': 'NestProtect',
 '18:b4:30:40:1e:c5': 'NestGuard',
 '18:b4:30:58:3d:6c': 'NestCamera',
 '18:b4:30:8c:03:e4': 'NestCamIQ',
 '20:df:b9:20:87:39': 'GoogleHomeMini',
 '30:52:cb:a3:4f:5f': 'RokuTV',
 '3c:f7:a4:f2:15:87': 'iPhone',
 '40:9f:38:92:40:13': 'Roomba',
 '44:73:d6:01:3d:fd': 'LogitechLogiCircle',
 '48:d6:d5:98:53:84': 'GoogleHome',
 '50:c7:bf:92:a6:4a': 'TP-LinkSmartWiFiLEDBulb',
 '54:4a:16:f9:54:18': 'InsteonHub',
 '5c:aa:fd:6c:e0:d4': 'Sonos',
 '64:52:99:97:f8:40': 'ChamberlainmyQGarageOpener',
 '74:c2:46:1b:8e:e2': 'AmazonEchoGen1',
 '7c:64:56:60:71:74': 'SamsungSmartTV',
 '7c:70:bc:5d:09:d1': 'Canary',
 '94:10:3e:5c:2e:31': 'BelkinWeMoCrockpot',
 '94:10:3e:cc:67:95': 'BelkinWeMoLink',
 '94:4a:0c:08:7e:72': 'MiCasaVerdeVeraLite',
 'a4:f1:e8:8d:b0:9e': 'AndroidTablet',
 'ac:3f:a4:70:4a:d6': 'PiperNV',
 'b0:4e:26:20:15:8a': 'TP-LinkWiFiPlug',
 'b0:7f:b9:a6:47:4d': 'NetgearArloCamera',
 'b0:c5:54:03:c7:09': 'D-LinkDCS-5009LCamera',
 'b4:79:a7:22:f9:fc': 'WinkHub',
 'c0:56:27:53:09:6d': 'BelkinNetcam',
 'c8:db:26:02:bb:bb': 'LogitechHarmonyHub',
 'cc:b8:a8:ad:4d:04': 'AugustDoorbellCam',
 'd0:03:4b:39:12:e3': 'AppleTV(4thGen)',
 'd0:52:a8:63:47:9e': 'SamsungSmartThingsHub',
 'd0:73:d5:12:84:d1': 'LIFXVirtualBulb',
 'd4:90:9c:cc:62:42': 'AppleHomePod',
 'd8:f7:10:c2:29:be': 'HarmonKardonInvoke',
 'e4:71:85:25:ce:ec': 'SecurifiAlmond',
 'e8:b2:ac:af:62:0f': 'iPad',
 'f4:5e:ab:5e:c0:23': 'CasetaWirelessHub',
 'f4:f2:6d:ce:9a:5d': 'GoogleOnHub',
# IoT devices captures MAC and Device names
'00:17:88:24:76:ff': 'Hue-Device',
 '00:1a:22:03:cb:be': 'MAXGateway',
 '00:1a:22:05:c4:2e': 'HomeMaticPlug',
 '00:24:e4:24:80:2a': 'Withings',
 '00:b5:6d:06:08:ba': 'unknown',
 '1c:5f:2b:aa:fd:4e': 'D-LinkDevice',
 '20:f8:5e:ca:91:52': 'Aria',
 '24:77:03:7c:ea:dc': 'unknown',
 '28:b2:bd:c3:41:79': 'unknown',
 '38:0b:40:ef:85:41': 'unknown',
 '3c:49:37:03:17:db': 'EdnetCam',
 '3c:49:37:03:17:f0': 'EdnetCam',
 '50:c7:bf:00:c7:03': 'TP-LinkPlugHS110',
 '50:c7:bf:00:fc:a3': 'TP-LinkPlugHS100',
 '5c:cf:7f:06:d9:02': 'iKettle2',
 '5c:cf:7f:07:ae:fb': 'SmarterCoffee',
 '6c:72:20:c5:17:5a': 'D-LinkWaterSensor',
 '74:da:38:23:22:7b': 'EdimaxPlug2101W',
 '74:da:38:4a:76:49': 'EdimaxPlug1101W',
 '74:da:38:80:79:fc': 'EdimaxCam',
 '74:da:38:80:7a:08': 'EdimaxCam',
 '84:18:26:7b:5f:6b': 'Lightify',
 '90:8d:78:a8:e1:43': 'D-LinkSensor',
 '90:8d:78:a9:3d:6f': 'D-LinkSwitch',
 '90:8d:78:dd:0d:60': 'D-LinkSiren',
 '94:10:3e:34:0c:b5': 'WeMoSwitch',
 '94:10:3e:35:01:c1': 'WeMoSwitch',
 '94:10:3e:41:c2:05': 'WeMoInsightSwitch',
 '94:10:3e:42:80:69': 'WeMoInsightSwitch',
 '94:10:3e:cd:37:65': 'WeMoLink',
 'ac:cf:23:62:3c:6e': 'EdnetGateway',
 'b0:c5:54:1c:71:85': 'D-LinkDayCam',
 'b0:c5:54:25:5b:0e': 'D-LinkCam',
 'bc:f5:ac:f4:c0:9d': 'unknown'}

In [6]:
len(MAC_list)

116

### Calculating the payload entropy value.


# ↓ 

In [7]:
def pre_entropy(payload):
    characters=[]
    for i in payload:
            characters.append(i)
    return shannon(characters)


def shannon(data):
    freq_dict={} 
    for i in data:
        if i in freq_dict:
            freq_dict[i] += 1
        else:
            freq_dict[i] = 1    
    entropy = 0.0
    logarithm_base = 2
    payload_size = len(data) #
    for key in freq_dict.keys():
        frequency = float(freq_dict[key])/payload_size
        if frequency > 0: 
            entropy = entropy + frequency * math.log(frequency, logarithm_base)
    return -entropy

### This section is the main backbone of our program. In this section, the following operations are performed briefly.


#### - The pcap_files variable contains the addresses of the pcap files. The `for` loop moves through the values of this variable, allowing all files to be processed.


#### - The second for loop examines individual packets in the processed pcap file. All features belonging to a packet are extracted and processed as a new line in the fingerprint file at the end of the second for loop.


#### - All properties are initially assigned a value of 0. These properties are then queried in the packet. If the properties have corresponding data, the data is processed in the variable, otherwise, the value of the variable remains as 0.

In [8]:
count=0
ths = open("./dataset/Main_Sentinel.csv", "w")
header="ARP,LLC,EAPOL,IP,ICMP,ICMP6,TCP,UDP,TCP_w_size,HTTP,HTTPS,DHCP,BOOTP,SSDP,DNS,MDNS,NTP,IP_padding,IP_add_count,IP_ralert,Portcl_src,Portcl_dst,Pck_size,Pck_rawdata,payload_l,Entropy,Label,MAC\n"
ths.write(header)
dst_ip_list={}

for i in MAC_list:
    dst_ip_list[i]=[]
import time

for i in files_add:
    print(i)
    pkt = PcapReader(i)
    print("\n\n"+"========"+ i[8:]+"========"+"\n" )
    print(pkt)

    for jj,j in enumerate(pkt):
      #  print(j)
        ip_add_count=0
        layer_2_arp = 0
        layer_2_llc = 0

        layer_3_eapol = 0
        layer_3_ip = 0
        layer_3_icmp = 0
        layer_3_icmp6 = 0



        layer_4_tcp = 0
        layer_4_udp = 0
        layer_4_tcp_ws=0


        layer_7_http = 0
        layer_7_https = 0
        layer_7_dhcp = 0
        layer_7_bootp = 0
        layer_7_ssdp = 0
        layer_7_dns = 0
        layer_7_mdns = 0
        layer_7_ntp = 0

        ip_padding = 0
        ip_ralert = 0


        port_class_src = 0
        port_class_dst = 0

        pck_size = 0
        pck_rawdata = 0
        entropy=0

        layer_4_payload_l=0

        try:

            pck_size=j.len

        except:pass

        try:

            if j[IP]:

                layer_3_ip = 1
            temp=str(j[IP].dst)
            if temp not in dst_ip_list[j.src]:
                dst_ip_list[j.src].append(temp)
            ip_add_count=len(dst_ip_list[j.src])

            port_class_src = port_class(j[IP].sport)
            port_class_dst = port_class(j[IP].dport)

        except:pass

        temp=str(j.show)

        if "ICMPv6" in temp:

            layer_3_icmp6 = 1

        try:
            if j[IP].ihl >5:
                if IPOption_Router_Alert(j):
                    pad=str(IPOption_Router_Alert(j).show)
                    if "Padding" in pad:
                        ip_padding=1
                    ip_ralert = 1
        except:pass

        if j.haslayer(ICMP):
            layer_3_icmp = 1


        if j.haslayer(Raw):
            pck_rawdata = 1

        if j.haslayer(UDP):

            layer_4_udp = 1
            if j[UDP].sport==68 or j[UDP].sport==67:
                layer_7_dhcp = 1
                layer_7_bootp = 1
            if j[UDP].sport==53 or j[UDP].dport==53:
                layer_7_dns = 1
            if j[UDP].sport==5353 or j[UDP].dport==5353:
                layer_7_mdns = 1
            if j[UDP].sport==1900 or j[UDP].dport==1900:
                layer_7_ssdp = 1
            if j[UDP].sport==123 or j[UDP].dport==123:
                layer_7_ntp = 1

        try:
            if j[UDP].payload:
                layer_4_payload_l=len(j[UDP].payload)
        except:pass



        if j.haslayer(TCP):
            layer_4_tcp = 1
            layer_4_tcp_ws=j[TCP].window
            if j[TCP].sport==80 or j[TCP].dport==80:
                layer_7_http = 1
            if j[TCP].sport==443 or j[TCP].dport==443:
                layer_7_https = 1
            try:
                if j[TCP].payload:
                    layer_4_payload_l=len(j[TCP].payload)
            except:pass

        if j.haslayer(ARP):
            layer_2_arp = 1

        if j.haslayer(LLC):
            layer_2_llc = 1

        if j.haslayer(EAPOL):
            layer_3_eapol = 1
        try: 
            entropy=pre_entropy(j[Raw].original)
        except:pass
        label=MAC_list[j.src]
        line=[layer_2_arp, layer_2_llc, layer_3_eapol, layer_3_ip, layer_3_icmp, layer_3_icmp6, layer_4_tcp, layer_4_udp, layer_4_tcp_ws, layer_7_http, layer_7_https, layer_7_dhcp, layer_7_bootp, layer_7_ssdp, layer_7_dns, layer_7_mdns, layer_7_ntp, ip_padding, ip_add_count, ip_ralert, port_class_src, port_class_dst, pck_size, pck_rawdata,layer_4_payload_l,entropy, label,j.src]  
        line=str(line).replace("[","")
        line=str(line).replace("]","")
        line=str(line).replace(", ",",")
        line=str(line).replace("\'","")
        if label!="unknown":
            ths.write(str(line)+"\n")  
    
ths.close()


./pcaps/HueSwitch/Setup-C-8-STA.pcap


========HueSwitch/Setup-C-8-STA.pcap========

./pcaps/HueSwitch/Setup-C-9-STA.pcap


========HueSwitch/Setup-C-9-STA.pcap========

./pcaps/HueSwitch/Setup-C-2-STA.pcap


========HueSwitch/Setup-C-2-STA.pcap========

./pcaps/HueSwitch/Setup-C-3-STA.pcap


========HueSwitch/Setup-C-3-STA.pcap========

./pcaps/HueSwitch/Setup-C-10-STA.pcap


========HueSwitch/Setup-C-10-STA.pcap========

./pcaps/HueSwitch/Setup-A-4-STA.pcap


========HueSwitch/Setup-A-4-STA.pcap========

./pcaps/HueSwitch/Setup-A-5-STA.pcap


========HueSwitch/Setup-A-5-STA.pcap========

./pcaps/HueSwitch/Setup-A-9-STA.pcap


========HueSwitch/Setup-A-9-STA.pcap========

./pcaps/HueSwitch/Setup-A-8-STA.pcap


========HueSwitch/Setup-A-8-STA.pcap========

./pcaps/HueSwitch/Setup-A-3-STA.pcap


========HueSwitch/Setup-A-3-STA.pcap========

./pcaps/HueSwitch/Setup-A-2-STA.pcap


========HueSwitch/Setup-A-2-STA.pcap========

./pcaps/HueSwitch/Setup-C-5-STA.pcap


========HueSwitch/Setup-

./pcaps/D-LinkDayCam/Setup-A-5-STA.pcap


========D-LinkDayCam/Setup-A-5-STA.pcap========

./pcaps/D-LinkDayCam/Setup-A-3-STA.pcap


========D-LinkDayCam/Setup-A-3-STA.pcap========

./pcaps/D-LinkDayCam/Setup-A-2-STA.pcap


========D-LinkDayCam/Setup-A-2-STA.pcap========

./pcaps/D-LinkDayCam/Setup-C-5-STA.pcap


========D-LinkDayCam/Setup-C-5-STA.pcap========

./pcaps/D-LinkDayCam/Setup-C-4-STA.pcap


========D-LinkDayCam/Setup-C-4-STA.pcap========

./pcaps/D-LinkDayCam/Setup-C-12-STA.pcap


========D-LinkDayCam/Setup-C-12-STA.pcap========

./pcaps/D-LinkDayCam/Setup-C-13-STA.pcap


========D-LinkDayCam/Setup-C-13-STA.pcap========

./pcaps/D-LinkDayCam/Setup-C-1-STA.pcap


========D-LinkDayCam/Setup-C-1-STA.pcap========

./pcaps/D-LinkDayCam/Setup-C-6-STA.pcap


========D-LinkDayCam/Setup-C-6-STA.pcap========

./pcaps/D-LinkDayCam/Setup-C-7-STA.pcap


========D-LinkDayCam/Setup-C-7-STA.pcap========

./pcaps/D-LinkDayCam/Setup-A-1-STA.pcap


========D-LinkDayCam/Setup-A-1-STA.pcap=====

./pcaps/Withings/Setup-C-4-STA.pcap


========Withings/Setup-C-4-STA.pcap========

./pcaps/Withings/Setup-C-18-STA.pcap


========Withings/Setup-C-18-STA.pcap========

./pcaps/Withings/Setup-C-19-STA.pcap


========Withings/Setup-C-19-STA.pcap========

./pcaps/Withings/Setup-C-12-STA.pcap


========Withings/Setup-C-12-STA.pcap========

./pcaps/Withings/Setup-C-13-STA.pcap


========Withings/Setup-C-13-STA.pcap========

./pcaps/Withings/Setup-C-1-STA.pcap


========Withings/Setup-C-1-STA.pcap========

./pcaps/Withings/Setup-C-6-STA.pcap


========Withings/Setup-C-6-STA.pcap========

./pcaps/Withings/Setup-C-7-STA.pcap


========Withings/Setup-C-7-STA.pcap========

./pcaps/Withings/Setup-A-1-STA.pcap


========Withings/Setup-A-1-STA.pcap========

./pcaps/Withings/Setup-C-15-STA.pcap


========Withings/Setup-C-15-STA.pcap========

./pcaps/Withings/Setup-C-14-STA.pcap


========Withings/Setup-C-14-STA.pcap========

./pcaps/D-LinkDoorSensor/Setup-C-8-STA.pcap


========D-LinkDoorSensor/Setu

./pcaps/WeMoLink/Setup-C-2-STA.pcap


========WeMoLink/Setup-C-2-STA.pcap========

./pcaps/WeMoLink/Setup-C-3-STA.pcap


========WeMoLink/Setup-C-3-STA.pcap========

./pcaps/WeMoLink/Setup-A-4-STA.pcap


========WeMoLink/Setup-A-4-STA.pcap========

./pcaps/WeMoLink/Setup-A-5-STA.pcap


========WeMoLink/Setup-A-5-STA.pcap========

./pcaps/WeMoLink/Setup-A-14-STA.pcap


========WeMoLink/Setup-A-14-STA.pcap========

./pcaps/WeMoLink/Setup-A-15-STA.pcap


========WeMoLink/Setup-A-15-STA.pcap========

./pcaps/WeMoLink/Setup-A-9-STA.pcap


========WeMoLink/Setup-A-9-STA.pcap========

./pcaps/WeMoLink/Setup-A-8-STA.pcap


========WeMoLink/Setup-A-8-STA.pcap========

./pcaps/WeMoLink/Setup-A-3-STA.pcap


========WeMoLink/Setup-A-3-STA.pcap========

./pcaps/WeMoLink/Setup-A-2-STA.pcap


========WeMoLink/Setup-A-2-STA.pcap========

./pcaps/WeMoLink/Setup-A-13-STA.pcap


========WeMoLink/Setup-A-13-STA.pcap========

./pcaps/WeMoLink/Setup-A-12-STA.pcap


========WeMoLink/Setup-A-12-STA.pcap======

./pcaps/TP-LinkPlugHS100/Setup-C-9-STA.pcap


========TP-LinkPlugHS100/Setup-C-9-STA.pcap========

./pcaps/TP-LinkPlugHS100/Setup-C-2-STA.pcap


========TP-LinkPlugHS100/Setup-C-2-STA.pcap========

./pcaps/TP-LinkPlugHS100/Setup-C-3-STA.pcap


========TP-LinkPlugHS100/Setup-C-3-STA.pcap========

./pcaps/TP-LinkPlugHS100/Setup-C-11-STA.pcap


========TP-LinkPlugHS100/Setup-C-11-STA.pcap========

./pcaps/TP-LinkPlugHS100/Setup-C-10-STA.pcap


========TP-LinkPlugHS100/Setup-C-10-STA.pcap========

./pcaps/TP-LinkPlugHS100/Setup-A-4-STA.pcap


========TP-LinkPlugHS100/Setup-A-4-STA.pcap========

./pcaps/TP-LinkPlugHS100/Setup-A-5-STA.pcap


========TP-LinkPlugHS100/Setup-A-5-STA.pcap========

./pcaps/TP-LinkPlugHS100/Setup-A-3-STA.pcap


========TP-LinkPlugHS100/Setup-A-3-STA.pcap========

./pcaps/TP-LinkPlugHS100/Setup-A-2-STA.pcap


========TP-LinkPlugHS100/Setup-A-2-STA.pcap========

./pcaps/TP-LinkPlugHS100/Setup-C-5-STA.pcap


========TP-LinkPlugHS100/Setup-C-5-STA.pcap========

./pcap

./pcaps/HomeMaticPlug/Setup-C-2-STA.pcap


========HomeMaticPlug/Setup-C-2-STA.pcap========

./pcaps/HomeMaticPlug/Setup-C-3-STA.pcap


========HomeMaticPlug/Setup-C-3-STA.pcap========

./pcaps/HomeMaticPlug/Setup-C-11-STA.pcap


========HomeMaticPlug/Setup-C-11-STA.pcap========

./pcaps/HomeMaticPlug/Setup-C-10-STA.pcap


========HomeMaticPlug/Setup-C-10-STA.pcap========

./pcaps/HomeMaticPlug/Setup-C-16-STA.pcap


========HomeMaticPlug/Setup-C-16-STA.pcap========

./pcaps/HomeMaticPlug/Setup-C-17-STA.pcap


========HomeMaticPlug/Setup-C-17-STA.pcap========

./pcaps/HomeMaticPlug/Setup-C-5-STA.pcap


========HomeMaticPlug/Setup-C-5-STA.pcap========

./pcaps/HomeMaticPlug/Setup-C-4-STA.pcap


========HomeMaticPlug/Setup-C-4-STA.pcap========

./pcaps/HomeMaticPlug/Setup-C-18-STA.pcap


========HomeMaticPlug/Setup-C-18-STA.pcap========

./pcaps/HomeMaticPlug/Setup-C-19-STA.pcap


========HomeMaticPlug/Setup-C-19-STA.pcap========

./pcaps/HomeMaticPlug/Setup-C-12-STA.pcap


========HomeMa

./pcaps/D-LinkWaterSensor/Setup-C-2-STA.pcap


========D-LinkWaterSensor/Setup-C-2-STA.pcap========

./pcaps/D-LinkWaterSensor/Setup-C-3-STA.pcap


========D-LinkWaterSensor/Setup-C-3-STA.pcap========

./pcaps/D-LinkWaterSensor/Setup-C-11-STA.pcap


========D-LinkWaterSensor/Setup-C-11-STA.pcap========

./pcaps/D-LinkWaterSensor/Setup-C-10-STA.pcap


========D-LinkWaterSensor/Setup-C-10-STA.pcap========

./pcaps/D-LinkWaterSensor/Setup-C-16-STA.pcap


========D-LinkWaterSensor/Setup-C-16-STA.pcap========

./pcaps/D-LinkWaterSensor/Setup-C-17-STA.pcap


========D-LinkWaterSensor/Setup-C-17-STA.pcap========

./pcaps/D-LinkWaterSensor/Setup-C-5-STA.pcap


========D-LinkWaterSensor/Setup-C-5-STA.pcap========

./pcaps/D-LinkWaterSensor/Setup-C-4-STA.pcap


========D-LinkWaterSensor/Setup-C-4-STA.pcap========

./pcaps/D-LinkWaterSensor/Setup-C-18-STA.pcap


========D-LinkWaterSensor/Setup-C-18-STA.pcap========

./pcaps/D-LinkWaterSensor/Setup-C-19-STA.pcap


========D-LinkWaterSensor/Setup-C

./pcaps/D-LinkSensor/Setup-A-5-STA.pcap


========D-LinkSensor/Setup-A-5-STA.pcap========

./pcaps/D-LinkSensor/Setup-A-3-STA.pcap


========D-LinkSensor/Setup-A-3-STA.pcap========

./pcaps/D-LinkSensor/Setup-A-2-STA.pcap


========D-LinkSensor/Setup-A-2-STA.pcap========

./pcaps/D-LinkSensor/Setup-C-5-STA.pcap


========D-LinkSensor/Setup-C-5-STA.pcap========

./pcaps/D-LinkSensor/Setup-C-4-STA.pcap


========D-LinkSensor/Setup-C-4-STA.pcap========

./pcaps/D-LinkSensor/Setup-C-1-STA.pcap


========D-LinkSensor/Setup-C-1-STA.pcap========

./pcaps/D-LinkSensor/Setup-B-3-STA.pcap


========D-LinkSensor/Setup-B-3-STA.pcap========

./pcaps/D-LinkSensor/Setup-B-2-STA.pcap


========D-LinkSensor/Setup-B-2-STA.pcap========

./pcaps/D-LinkSensor/Setup-C-6-STA.pcap


========D-LinkSensor/Setup-C-6-STA.pcap========

./pcaps/D-LinkSensor/Setup-C-7-STA.pcap


========D-LinkSensor/Setup-C-7-STA.pcap========

./pcaps/D-LinkSensor/Setup-B-4-STA.pcap


========D-LinkSensor/Setup-B-4-STA.pcap========


./pcaps/D-LinkSiren/Setup-C-3-STA.pcap


========D-LinkSiren/Setup-C-3-STA.pcap========

./pcaps/D-LinkSiren/Setup-C-11-STA.pcap


========D-LinkSiren/Setup-C-11-STA.pcap========

./pcaps/D-LinkSiren/Setup-C-10-STA.pcap


========D-LinkSiren/Setup-C-10-STA.pcap========

./pcaps/D-LinkSiren/Setup-C-16-STA.pcap


========D-LinkSiren/Setup-C-16-STA.pcap========

./pcaps/D-LinkSiren/Setup-C-17-STA.pcap


========D-LinkSiren/Setup-C-17-STA.pcap========

./pcaps/D-LinkSiren/Setup-C-5-STA.pcap


========D-LinkSiren/Setup-C-5-STA.pcap========

./pcaps/D-LinkSiren/Setup-C-4-STA.pcap


========D-LinkSiren/Setup-C-4-STA.pcap========

./pcaps/D-LinkSiren/Setup-C-18-STA.pcap


========D-LinkSiren/Setup-C-18-STA.pcap========

./pcaps/D-LinkSiren/Setup-C-19-STA.pcap


========D-LinkSiren/Setup-C-19-STA.pcap========

./pcaps/D-LinkSiren/Setup-C-12-STA.pcap


========D-LinkSiren/Setup-C-12-STA.pcap========

./pcaps/D-LinkSiren/Setup-C-13-STA.pcap


========D-LinkSiren/Setup-C-13-STA.pcap========

./pca

./pcaps/HueBridge/Setup-C-7-STA.pcap


========HueBridge/Setup-C-7-STA.pcap========

./pcaps/HueBridge/Setup-A-1-STA.pcap


========HueBridge/Setup-A-1-STA.pcap========

./pcaps/HueBridge/Setup-C-15-STA.pcap


========HueBridge/Setup-C-15-STA.pcap========

./pcaps/HueBridge/Setup-C-14-STA.pcap


========HueBridge/Setup-C-14-STA.pcap========



### Input & Output

`Input Files`: MAIN_Sentinel.csv

`Output Files`: IPAssess.csv

In [2]:
dataset="dataset/Main_Sentinel.csv"
df=pd.read_csv(dataset)

In [3]:
df

,ARP,LLC,EAPOL,IP,ICMP,ICMP6,TCP,UDP,TCP_w_size,HTTP,...,IP_add_count,IP_ralert,Portcl_src,Portcl_dst,Pck_size,Pck_rawdata,payload_l,Entropy,Label,MAC
0,0,0,0,1,0,0,1,0,3754,1,...,1,0,1,2,52,0,0,0.000000,Hue-Device,00:17:88:24:76:ff
1,0,0,0,1,0,0,1,0,3754,1,...,1,0,1,2,69,1,17,3.616875,Hue-Device,00:17:88:24:76:ff
2,0,0,0,1,0,0,1,0,3754,1,...,1,0,1,2,920,1,868,5.175318,Hue-Device,00:17:88:24:76:ff
3,0,0,0,1,0,0,1,0,3754,1,...,1,0,1,2,52,0,0,0.000000,Hue-Device,00:17:88:24:76:ff
4,0,0,0,1,0,0,1,0,28960,1,...,1,0,1,3,60,0,0,0.000000,Hue-Device,00:17:88:24:76:ff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107647,0,0,0,1,0,0,1,0,3784,1,...,92,0,2,1,52,0,0,0.000000,Hue-Device,00:17:88:24:76:ff
107648,0,0,0,1,0,0,0,1,0,0,...,92,0,2,2,337,1,309,5.472835,Hue-Device,00:17:88:24:76:ff
107649,0,0,0,1,0,0,0,1,0,0,...,92,0,2,2,337,1,309,5.472835,Hue-Device,00:17:88:24:76:ff
107650,0,0,0,1,0,0,0,1,0,0,...,92,0,3,1,76,0,48,0.000000,Hue-Device,00:17:88:24:76:ff


## Correlation matrix on Main Feature Set i.e FP_Main

In [4]:
dataframe1 = df
del dataframe1["MAC"]
del dataframe1["Label"]
matrix = dataframe1.corr()
matrix = matrix['IP']
matrix

ARP            -0.704634
LLC            -0.164071
EAPOL          -0.471234
IP              1.000000
ICMP            0.014350
ICMP6          -0.405104
TCP             0.325868
UDP             0.103535
TCP_w_size      0.135689
HTTP            0.182248
HTTPS           0.096840
DHCP            0.025053
BOOTP           0.025053
SSDP            0.048330
DNS             0.046247
MDNS            0.036154
NTP             0.027813
IP_padding           NaN
IP_add_count    0.232252
IP_ralert            NaN
Portcl_src      0.505409
Portcl_dst      0.450926
Pck_size        0.121116
Pck_rawdata     0.038140
payload_l       0.109698
Entropy         0.090195
Name: IP, dtype: float64

In [12]:
df_IOT_Sentinel=matrix[(matrix['IP']<=(-0.05)) | (matrix['IP']>=0.05)]
df_IOT_Sentinel

,ARP,LLC,EAPOL,IP,ICMP,ICMP6,TCP,UDP,TCP_w_size,HTTP,...,NTP,IP_padding,IP_add_count,IP_ralert,Portcl_src,Portcl_dst,Pck_size,Pck_rawdata,payload_l,Entropy
ARP,1.000000,-0.005992,-0.017211,-0.704634,-0.010112,-0.014796,-0.229618,-0.095085,-0.095611,-0.128418,...,-0.019598,NaN,-0.163653,NaN,-0.356129,-0.317738,-0.105607,-0.117829,-0.085674,-0.106495
LLC,-0.005992,1.000000,-0.004008,-0.164071,-0.002354,-0.003445,-0.053466,-0.022140,-0.022263,-0.029902,...,-0.004563,NaN,-0.038106,NaN,-0.082923,-0.073984,-0.023930,0.050857,-0.019949,-0.002194
EAPOL,-0.017211,-0.004008,1.000000,-0.471234,-0.006762,-0.009895,-0.153560,-0.063590,-0.063941,-0.085881,...,-0.013107,NaN,-0.109445,NaN,-0.238166,-0.212492,-0.037198,0.146069,-0.057296,0.045193
IP,-0.704634,-0.164071,-0.471234,1.000000,0.014350,-0.405104,0.325868,0.103535,0.135689,0.182248,...,0.027813,NaN,0.232252,NaN,0.505409,0.450926,0.121116,0.038140,0.109698,0.090195
ICMP6,-0.014796,-0.003445,-0.009895,-0.405104,-0.005813,1.000000,-0.132010,-0.054666,-0.054968,-0.073829,...,-0.011267,NaN,-0.094086,NaN,-0.204743,-0.182672,-0.060601,-0.067741,-0.049255,-0.061225
TCP,-0.229618,-0.053466,-0.153560,0.325868,-0.090218,-0.132010,1.000000,-0.848363,0.416393,0.559267,...,-0.174857,NaN,0.168494,NaN,0.107414,0.225689,0.076780,-0.007594,0.050786,0.073819
UDP,-0.095085,-0.022140,-0.063590,0.103535,-0.037359,-0.054666,-0.848363,1.000000,-0.353252,-0.474462,...,0.206111,NaN,-0.075691,NaN,0.227492,0.063695,0.003391,-0.040144,0.028475,-0.044473
TCP_w_size,-0.095611,-0.022263,-0.063941,0.135689,-0.037566,-0.054968,0.416393,-0.353252,1.000000,0.220188,...,-0.072809,NaN,0.245606,NaN,0.090767,0.033248,-0.046144,-0.169906,-0.062262,-0.104930
HTTP,-0.128418,-0.029902,-0.085881,0.182248,-0.050456,-0.073829,0.559267,-0.474462,0.220188,1.000000,...,-0.097792,NaN,-0.089440,NaN,-0.225665,0.252279,-0.102774,-0.047394,-0.121421,-0.112212
HTTPS,-0.068237,-0.015889,-0.045634,0.096840,-0.026810,-0.039230,0.297175,-0.252113,0.184261,-0.340492,...,-0.051963,NaN,0.037997,NaN,0.313862,-0.300200,-0.079916,-0.090033,-0.086300,-0.016497


## Create IPAssess


In [13]:
df=pd.read_csv("dataset/Main_Sentinel.csv")
deleted=["SSDP","DNS","MDNS","Pck_rawdata","NTP","DHCP","BOOTP","ICMP","IP_padding","IP_ralert"]
name="IPAssess_Sentinel.csv"
df=df.drop(columns=deleted)
df.to_csv('dataset/'+name, index=False)
df.columns

Index(['ARP', 'LLC', 'EAPOL', 'IP', 'ICMP6', 'TCP', 'UDP', 'TCP_w_size',
       'HTTP', 'HTTPS', 'IP_add_count', 'Portcl_src', 'Portcl_dst', 'Pck_size',
       'payload_l', 'Entropy', 'Label', 'MAC'],
      dtype='object')

In [14]:
len(df.columns)  # includes label, Mac

18

### Create Main Feature set without IP_padding,IP_alert

In [15]:
df=pd.read_csv("dataset/Main_Sentinel.csv")
deleted=["IP_padding","IP_ralert"]
name="Main_Sentinel_padding_ralert.csv"
df=df.drop(columns=deleted)
df.to_csv('dataset/'+name, index=False)
df.columns

Index(['ARP', 'LLC', 'EAPOL', 'IP', 'ICMP', 'ICMP6', 'TCP', 'UDP',
       'TCP_w_size', 'HTTP', 'HTTPS', 'DHCP', 'BOOTP', 'SSDP', 'DNS', 'MDNS',
       'NTP', 'IP_add_count', 'Portcl_src', 'Portcl_dst', 'Pck_size',
       'Pck_rawdata', 'payload_l', 'Entropy', 'Label', 'MAC'],
      dtype='object')

In [16]:
len(df.columns) # includes label, Mac

26

### Threshold 0.095

In [17]:
df_IOT_Sentinel=matrix[(matrix['IP']<=(-0.095)) | (matrix['IP']>=0.095)]
df_IOT_Sentinel

,ARP,LLC,EAPOL,IP,ICMP,ICMP6,TCP,UDP,TCP_w_size,HTTP,...,NTP,IP_padding,IP_add_count,IP_ralert,Portcl_src,Portcl_dst,Pck_size,Pck_rawdata,payload_l,Entropy
ARP,1.000000,-0.005992,-0.017211,-0.704634,-0.010112,-0.014796,-0.229618,-0.095085,-0.095611,-0.128418,...,-0.019598,NaN,-0.163653,NaN,-0.356129,-0.317738,-0.105607,-0.117829,-0.085674,-0.106495
LLC,-0.005992,1.000000,-0.004008,-0.164071,-0.002354,-0.003445,-0.053466,-0.022140,-0.022263,-0.029902,...,-0.004563,NaN,-0.038106,NaN,-0.082923,-0.073984,-0.023930,0.050857,-0.019949,-0.002194
EAPOL,-0.017211,-0.004008,1.000000,-0.471234,-0.006762,-0.009895,-0.153560,-0.063590,-0.063941,-0.085881,...,-0.013107,NaN,-0.109445,NaN,-0.238166,-0.212492,-0.037198,0.146069,-0.057296,0.045193
IP,-0.704634,-0.164071,-0.471234,1.000000,0.014350,-0.405104,0.325868,0.103535,0.135689,0.182248,...,0.027813,NaN,0.232252,NaN,0.505409,0.450926,0.121116,0.038140,0.109698,0.090195
ICMP6,-0.014796,-0.003445,-0.009895,-0.405104,-0.005813,1.000000,-0.132010,-0.054666,-0.054968,-0.073829,...,-0.011267,NaN,-0.094086,NaN,-0.204743,-0.182672,-0.060601,-0.067741,-0.049255,-0.061225
TCP,-0.229618,-0.053466,-0.153560,0.325868,-0.090218,-0.132010,1.000000,-0.848363,0.416393,0.559267,...,-0.174857,NaN,0.168494,NaN,0.107414,0.225689,0.076780,-0.007594,0.050786,0.073819
UDP,-0.095085,-0.022140,-0.063590,0.103535,-0.037359,-0.054666,-0.848363,1.000000,-0.353252,-0.474462,...,0.206111,NaN,-0.075691,NaN,0.227492,0.063695,0.003391,-0.040144,0.028475,-0.044473
TCP_w_size,-0.095611,-0.022263,-0.063941,0.135689,-0.037566,-0.054968,0.416393,-0.353252,1.000000,0.220188,...,-0.072809,NaN,0.245606,NaN,0.090767,0.033248,-0.046144,-0.169906,-0.062262,-0.104930
HTTP,-0.128418,-0.029902,-0.085881,0.182248,-0.050456,-0.073829,0.559267,-0.474462,0.220188,1.000000,...,-0.097792,NaN,-0.089440,NaN,-0.225665,0.252279,-0.102774,-0.047394,-0.121421,-0.112212
HTTPS,-0.068237,-0.015889,-0.045634,0.096840,-0.026810,-0.039230,0.297175,-0.252113,0.184261,-0.340492,...,-0.051963,NaN,0.037997,NaN,0.313862,-0.300200,-0.079916,-0.090033,-0.086300,-0.016497


In [18]:
df=pd.read_csv("dataset/Main_Sentinel.csv")
deleted=["SSDP","DNS","MDNS","Pck_rawdata","NTP","DHCP","BOOTP","ICMP","IP_padding","IP_ralert","Entropy"]
name="Threshold_0.095.csv"
df=df.drop(columns=deleted)
df.to_csv('dataset/'+name, index=False)
df.columns

Index(['ARP', 'LLC', 'EAPOL', 'IP', 'ICMP6', 'TCP', 'UDP', 'TCP_w_size',
       'HTTP', 'HTTPS', 'IP_add_count', 'Portcl_src', 'Portcl_dst', 'Pck_size',
       'payload_l', 'Label', 'MAC'],
      dtype='object')

### Threshold 0.1 

In [19]:
df_IOT_Sentinel=matrix[(matrix['IP']<=(-0.1)) | (matrix['IP']>=0.1)]
df_IOT_Sentinel

,ARP,LLC,EAPOL,IP,ICMP,ICMP6,TCP,UDP,TCP_w_size,HTTP,...,NTP,IP_padding,IP_add_count,IP_ralert,Portcl_src,Portcl_dst,Pck_size,Pck_rawdata,payload_l,Entropy
ARP,1.000000,-0.005992,-0.017211,-0.704634,-0.010112,-0.014796,-0.229618,-0.095085,-0.095611,-0.128418,...,-0.019598,NaN,-0.163653,NaN,-0.356129,-0.317738,-0.105607,-0.117829,-0.085674,-0.106495
LLC,-0.005992,1.000000,-0.004008,-0.164071,-0.002354,-0.003445,-0.053466,-0.022140,-0.022263,-0.029902,...,-0.004563,NaN,-0.038106,NaN,-0.082923,-0.073984,-0.023930,0.050857,-0.019949,-0.002194
EAPOL,-0.017211,-0.004008,1.000000,-0.471234,-0.006762,-0.009895,-0.153560,-0.063590,-0.063941,-0.085881,...,-0.013107,NaN,-0.109445,NaN,-0.238166,-0.212492,-0.037198,0.146069,-0.057296,0.045193
IP,-0.704634,-0.164071,-0.471234,1.000000,0.014350,-0.405104,0.325868,0.103535,0.135689,0.182248,...,0.027813,NaN,0.232252,NaN,0.505409,0.450926,0.121116,0.038140,0.109698,0.090195
ICMP6,-0.014796,-0.003445,-0.009895,-0.405104,-0.005813,1.000000,-0.132010,-0.054666,-0.054968,-0.073829,...,-0.011267,NaN,-0.094086,NaN,-0.204743,-0.182672,-0.060601,-0.067741,-0.049255,-0.061225
TCP,-0.229618,-0.053466,-0.153560,0.325868,-0.090218,-0.132010,1.000000,-0.848363,0.416393,0.559267,...,-0.174857,NaN,0.168494,NaN,0.107414,0.225689,0.076780,-0.007594,0.050786,0.073819
UDP,-0.095085,-0.022140,-0.063590,0.103535,-0.037359,-0.054666,-0.848363,1.000000,-0.353252,-0.474462,...,0.206111,NaN,-0.075691,NaN,0.227492,0.063695,0.003391,-0.040144,0.028475,-0.044473
TCP_w_size,-0.095611,-0.022263,-0.063941,0.135689,-0.037566,-0.054968,0.416393,-0.353252,1.000000,0.220188,...,-0.072809,NaN,0.245606,NaN,0.090767,0.033248,-0.046144,-0.169906,-0.062262,-0.104930
HTTP,-0.128418,-0.029902,-0.085881,0.182248,-0.050456,-0.073829,0.559267,-0.474462,0.220188,1.000000,...,-0.097792,NaN,-0.089440,NaN,-0.225665,0.252279,-0.102774,-0.047394,-0.121421,-0.112212
IP_add_count,-0.163653,-0.038106,-0.109445,0.232252,0.076822,-0.094086,0.168494,-0.075691,0.245606,-0.089440,...,0.079171,NaN,1.000000,NaN,0.182107,0.015900,0.126912,-0.004745,0.120910,0.062186


In [20]:
df=pd.read_csv("dataset/Main_Sentinel.csv")
deleted=["SSDP","DNS","MDNS","Pck_rawdata","NTP","DHCP","BOOTP","ICMP","IP_padding","IP_ralert","Entropy","HTTPS"]
name="Threshold_0.1.csv"
df=df.drop(columns=deleted)
df.to_csv('dataset/'+name, index=False)
df.columns

Index(['ARP', 'LLC', 'EAPOL', 'IP', 'ICMP6', 'TCP', 'UDP', 'TCP_w_size',
       'HTTP', 'IP_add_count', 'Portcl_src', 'Portcl_dst', 'Pck_size',
       'payload_l', 'Label', 'MAC'],
      dtype='object')

### Threshold 0.105

In [21]:
df_IOT_Sentinel=matrix[(matrix['IP']<=(-0.105)) | (matrix['IP']>=0.105)]
df_IOT_Sentinel

,ARP,LLC,EAPOL,IP,ICMP,ICMP6,TCP,UDP,TCP_w_size,HTTP,...,NTP,IP_padding,IP_add_count,IP_ralert,Portcl_src,Portcl_dst,Pck_size,Pck_rawdata,payload_l,Entropy
ARP,1.000000,-0.005992,-0.017211,-0.704634,-0.010112,-0.014796,-0.229618,-0.095085,-0.095611,-0.128418,...,-0.019598,NaN,-0.163653,NaN,-0.356129,-0.317738,-0.105607,-0.117829,-0.085674,-0.106495
LLC,-0.005992,1.000000,-0.004008,-0.164071,-0.002354,-0.003445,-0.053466,-0.022140,-0.022263,-0.029902,...,-0.004563,NaN,-0.038106,NaN,-0.082923,-0.073984,-0.023930,0.050857,-0.019949,-0.002194
EAPOL,-0.017211,-0.004008,1.000000,-0.471234,-0.006762,-0.009895,-0.153560,-0.063590,-0.063941,-0.085881,...,-0.013107,NaN,-0.109445,NaN,-0.238166,-0.212492,-0.037198,0.146069,-0.057296,0.045193
IP,-0.704634,-0.164071,-0.471234,1.000000,0.014350,-0.405104,0.325868,0.103535,0.135689,0.182248,...,0.027813,NaN,0.232252,NaN,0.505409,0.450926,0.121116,0.038140,0.109698,0.090195
ICMP6,-0.014796,-0.003445,-0.009895,-0.405104,-0.005813,1.000000,-0.132010,-0.054666,-0.054968,-0.073829,...,-0.011267,NaN,-0.094086,NaN,-0.204743,-0.182672,-0.060601,-0.067741,-0.049255,-0.061225
TCP,-0.229618,-0.053466,-0.153560,0.325868,-0.090218,-0.132010,1.000000,-0.848363,0.416393,0.559267,...,-0.174857,NaN,0.168494,NaN,0.107414,0.225689,0.076780,-0.007594,0.050786,0.073819
TCP_w_size,-0.095611,-0.022263,-0.063941,0.135689,-0.037566,-0.054968,0.416393,-0.353252,1.000000,0.220188,...,-0.072809,NaN,0.245606,NaN,0.090767,0.033248,-0.046144,-0.169906,-0.062262,-0.104930
HTTP,-0.128418,-0.029902,-0.085881,0.182248,-0.050456,-0.073829,0.559267,-0.474462,0.220188,1.000000,...,-0.097792,NaN,-0.089440,NaN,-0.225665,0.252279,-0.102774,-0.047394,-0.121421,-0.112212
IP_add_count,-0.163653,-0.038106,-0.109445,0.232252,0.076822,-0.094086,0.168494,-0.075691,0.245606,-0.089440,...,0.079171,NaN,1.000000,NaN,0.182107,0.015900,0.126912,-0.004745,0.120910,0.062186
Portcl_src,-0.356129,-0.082923,-0.238166,0.505409,-0.126898,-0.204743,0.107414,0.227492,0.090767,-0.225665,...,0.062860,NaN,0.182107,NaN,1.000000,-0.054303,0.003902,-0.097836,0.008523,0.029633


In [22]:
df=pd.read_csv("dataset/Main_Sentinel.csv")
deleted=["SSDP","DNS","MDNS","Pck_rawdata","NTP","DHCP","BOOTP","ICMP","IP_padding","IP_ralert","Entropy","HTTPS","UDP"]
name="Threshold_0.105.csv"
df=df.drop(columns=deleted)
df.to_csv('dataset/'+name, index=False)
df.columns

Index(['ARP', 'LLC', 'EAPOL', 'IP', 'ICMP6', 'TCP', 'TCP_w_size', 'HTTP',
       'IP_add_count', 'Portcl_src', 'Portcl_dst', 'Pck_size', 'payload_l',
       'Label', 'MAC'],
      dtype='object')

### Threshold 0.11

In [23]:
df_IOT_Sentinel=matrix[(matrix['IP']<=(-0.11)) | (matrix['IP']>=0.11)]
df_IOT_Sentinel

,ARP,LLC,EAPOL,IP,ICMP,ICMP6,TCP,UDP,TCP_w_size,HTTP,...,NTP,IP_padding,IP_add_count,IP_ralert,Portcl_src,Portcl_dst,Pck_size,Pck_rawdata,payload_l,Entropy
ARP,1.000000,-0.005992,-0.017211,-0.704634,-0.010112,-0.014796,-0.229618,-0.095085,-0.095611,-0.128418,...,-0.019598,NaN,-0.163653,NaN,-0.356129,-0.317738,-0.105607,-0.117829,-0.085674,-0.106495
LLC,-0.005992,1.000000,-0.004008,-0.164071,-0.002354,-0.003445,-0.053466,-0.022140,-0.022263,-0.029902,...,-0.004563,NaN,-0.038106,NaN,-0.082923,-0.073984,-0.023930,0.050857,-0.019949,-0.002194
EAPOL,-0.017211,-0.004008,1.000000,-0.471234,-0.006762,-0.009895,-0.153560,-0.063590,-0.063941,-0.085881,...,-0.013107,NaN,-0.109445,NaN,-0.238166,-0.212492,-0.037198,0.146069,-0.057296,0.045193
IP,-0.704634,-0.164071,-0.471234,1.000000,0.014350,-0.405104,0.325868,0.103535,0.135689,0.182248,...,0.027813,NaN,0.232252,NaN,0.505409,0.450926,0.121116,0.038140,0.109698,0.090195
ICMP6,-0.014796,-0.003445,-0.009895,-0.405104,-0.005813,1.000000,-0.132010,-0.054666,-0.054968,-0.073829,...,-0.011267,NaN,-0.094086,NaN,-0.204743,-0.182672,-0.060601,-0.067741,-0.049255,-0.061225
TCP,-0.229618,-0.053466,-0.153560,0.325868,-0.090218,-0.132010,1.000000,-0.848363,0.416393,0.559267,...,-0.174857,NaN,0.168494,NaN,0.107414,0.225689,0.076780,-0.007594,0.050786,0.073819
TCP_w_size,-0.095611,-0.022263,-0.063941,0.135689,-0.037566,-0.054968,0.416393,-0.353252,1.000000,0.220188,...,-0.072809,NaN,0.245606,NaN,0.090767,0.033248,-0.046144,-0.169906,-0.062262,-0.104930
HTTP,-0.128418,-0.029902,-0.085881,0.182248,-0.050456,-0.073829,0.559267,-0.474462,0.220188,1.000000,...,-0.097792,NaN,-0.089440,NaN,-0.225665,0.252279,-0.102774,-0.047394,-0.121421,-0.112212
IP_add_count,-0.163653,-0.038106,-0.109445,0.232252,0.076822,-0.094086,0.168494,-0.075691,0.245606,-0.089440,...,0.079171,NaN,1.000000,NaN,0.182107,0.015900,0.126912,-0.004745,0.120910,0.062186
Portcl_src,-0.356129,-0.082923,-0.238166,0.505409,-0.126898,-0.204743,0.107414,0.227492,0.090767,-0.225665,...,0.062860,NaN,0.182107,NaN,1.000000,-0.054303,0.003902,-0.097836,0.008523,0.029633


In [24]:
df=pd.read_csv("dataset/Main_Sentinel.csv")
deleted=["SSDP","DNS","MDNS","Pck_rawdata","NTP","DHCP","BOOTP","ICMP","IP_padding","IP_ralert","Entropy","HTTPS","UDP","payload_l"]
name="Threshold_0.11.csv"
df=df.drop(columns=deleted)
df.to_csv('dataset/'+name, index=False)
df.columns

Index(['ARP', 'LLC', 'EAPOL', 'IP', 'ICMP6', 'TCP', 'TCP_w_size', 'HTTP',
       'IP_add_count', 'Portcl_src', 'Portcl_dst', 'Pck_size', 'Label', 'MAC'],
      dtype='object')

### Threshold 0.125

In [25]:
df_IOT_Sentinel=matrix[(matrix['IP']<=(-0.125)) | (matrix['IP']>=0.125)]
df_IOT_Sentinel

,ARP,LLC,EAPOL,IP,ICMP,ICMP6,TCP,UDP,TCP_w_size,HTTP,...,NTP,IP_padding,IP_add_count,IP_ralert,Portcl_src,Portcl_dst,Pck_size,Pck_rawdata,payload_l,Entropy
ARP,1.000000,-0.005992,-0.017211,-0.704634,-0.010112,-0.014796,-0.229618,-0.095085,-0.095611,-0.128418,...,-0.019598,NaN,-0.163653,NaN,-0.356129,-0.317738,-0.105607,-0.117829,-0.085674,-0.106495
LLC,-0.005992,1.000000,-0.004008,-0.164071,-0.002354,-0.003445,-0.053466,-0.022140,-0.022263,-0.029902,...,-0.004563,NaN,-0.038106,NaN,-0.082923,-0.073984,-0.023930,0.050857,-0.019949,-0.002194
EAPOL,-0.017211,-0.004008,1.000000,-0.471234,-0.006762,-0.009895,-0.153560,-0.063590,-0.063941,-0.085881,...,-0.013107,NaN,-0.109445,NaN,-0.238166,-0.212492,-0.037198,0.146069,-0.057296,0.045193
IP,-0.704634,-0.164071,-0.471234,1.000000,0.014350,-0.405104,0.325868,0.103535,0.135689,0.182248,...,0.027813,NaN,0.232252,NaN,0.505409,0.450926,0.121116,0.038140,0.109698,0.090195
ICMP6,-0.014796,-0.003445,-0.009895,-0.405104,-0.005813,1.000000,-0.132010,-0.054666,-0.054968,-0.073829,...,-0.011267,NaN,-0.094086,NaN,-0.204743,-0.182672,-0.060601,-0.067741,-0.049255,-0.061225
TCP,-0.229618,-0.053466,-0.153560,0.325868,-0.090218,-0.132010,1.000000,-0.848363,0.416393,0.559267,...,-0.174857,NaN,0.168494,NaN,0.107414,0.225689,0.076780,-0.007594,0.050786,0.073819
TCP_w_size,-0.095611,-0.022263,-0.063941,0.135689,-0.037566,-0.054968,0.416393,-0.353252,1.000000,0.220188,...,-0.072809,NaN,0.245606,NaN,0.090767,0.033248,-0.046144,-0.169906,-0.062262,-0.104930
HTTP,-0.128418,-0.029902,-0.085881,0.182248,-0.050456,-0.073829,0.559267,-0.474462,0.220188,1.000000,...,-0.097792,NaN,-0.089440,NaN,-0.225665,0.252279,-0.102774,-0.047394,-0.121421,-0.112212
IP_add_count,-0.163653,-0.038106,-0.109445,0.232252,0.076822,-0.094086,0.168494,-0.075691,0.245606,-0.089440,...,0.079171,NaN,1.000000,NaN,0.182107,0.015900,0.126912,-0.004745,0.120910,0.062186
Portcl_src,-0.356129,-0.082923,-0.238166,0.505409,-0.126898,-0.204743,0.107414,0.227492,0.090767,-0.225665,...,0.062860,NaN,0.182107,NaN,1.000000,-0.054303,0.003902,-0.097836,0.008523,0.029633


In [26]:
df=pd.read_csv("dataset/Main_Sentinel.csv")
deleted=["SSDP","DNS","MDNS","Pck_rawdata","NTP","DHCP","BOOTP","ICMP","IP_padding","IP_ralert","Entropy","HTTPS","UDP","payload_l","Pck_size"]
name="Threshold_0.125.csv"
df=df.drop(columns=deleted)
df.to_csv('dataset/'+name, index=False)
df.columns

Index(['ARP', 'LLC', 'EAPOL', 'IP', 'ICMP6', 'TCP', 'TCP_w_size', 'HTTP',
       'IP_add_count', 'Portcl_src', 'Portcl_dst', 'Label', 'MAC'],
      dtype='object')